Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 8
  - _**[Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/)**_ — Ben Gorman
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) — Alex Rogozhnikov
  - [How to explain gradient boosting](https://explained.ai/gradient-boosting/) — Terence Parr & Jeremy Howard

In [1]:
# Imports 
import sys
!pip install pandas-profiling==2.*
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, mean_tweedie_deviance
from sklearn.metrics import explained_variance_score, max_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
from sklearn.feature_selection import SelectKBest, f_regression

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
TPPI = pd.read_csv('Total_Population_Period_Indicators')
df = TPPI.dropna(subset=['Births'])

## **Exploratory Analysis:**

In [3]:
# pandas profile report for original dataset
profile = ProfileReport(df, minimal= True).to_notebook_iframe()
profile

In [4]:
df.describe()

,Unnamed: 0,LocID,MidPeriod,Total Fertility,NRR(surviving daughters),Crude Birth Rate,Births,Life Expectancy(birth),LExMale,LExFemale,Infant Mortality Rate,Under-five Mortality,Crude Death Rate,Deaths,DeathsMale,DeathsFemale,Net Migration Rate(pK),Net Migrants(K),GrowthRate,Natural Increase Rate,Sex ratio(m per f births,Mean Age Childbearing(f),PopFemale,PopMale,PopTotal,PopDensity
count,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7.088000e+03,7.088000e+03,7.088000e+03,7088.000000
mean,3814.994357,1077.688488,1994.875000,3.831392,1.534372,27.639051,57279.246641,64.279876,62.067798,66.536494,59.517827,87.951534,11.360619,24977.727445,13181.393661,11796.333783,-0.130939,-214.672219,1.617724,16.278433,1.053711,28.810993,2.452477e+05,2.489027e+05,4.941504e+05,141.007502
std,2210.800671,729.124394,33.862164,1.897975,0.604770,12.934401,119362.543037,12.607073,12.158732,13.112974,52.344031,85.371346,5.500324,53517.256616,28209.979282,25322.540725,7.185746,2824.850320,1.247379,10.526515,0.017371,1.415274,5.437541e+05,5.560014e+05,1.099675e+06,827.146104
min,0.000000,4.000000,1953.000000,0.850000,0.410000,5.558000,1.198000,14.490000,11.880000,18.120000,0.126000,0.231000,1.147000,1.597000,0.808000,0.681000,-70.787000,-24462.517000,-5.321000,-20.979000,1.004000,23.897000,7.923000e+00,9.321000e+00,1.724400e+01,0.070000
25%,1915.750000,462.000000,1971.750000,2.079750,0.970000,15.911750,602.718250,55.340000,53.580000,57.050000,16.031250,19.552750,7.508500,287.015000,150.790000,136.603250,-1.400000,-457.890000,0.746000,7.604750,1.050000,27.712000,2.499810e+03,2.478078e+03,4.949038e+03,17.057500
50%,3783.500000,922.000000,1990.500000,3.301000,1.457500,25.893000,5639.506000,66.565000,64.045000,69.240000,43.623500,56.928500,9.880500,2716.019000,1410.282000,1289.021000,-0.239000,-17.073000,1.663000,17.738000,1.050000,28.997000,2.588380e+04,2.529137e+04,5.135256e+04,37.360000
75%,5779.250000,1596.000000,2009.250000,5.642000,2.039000,39.644750,51853.495500,73.340000,70.500000,76.290000,93.709250,137.602000,13.356750,20751.778250,11056.459000,9867.950000,0.859250,88.075000,2.431000,25.183500,1.060000,29.703250,2.122962e+05,2.065572e+05,4.184542e+05,102.086000
max,7631.000000,5501.000000,2098.000000,8.800000,3.653000,58.263000,701277.931000,94.020000,91.140000,96.930000,319.239000,465.517000,61.634000,606048.782000,313187.571000,292861.211000,134.414000,23278.467000,16.986000,42.294000,1.173000,34.997000,5.425119e+06,5.443229e+06,1.086835e+07,33470.669000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7088 entries, 0 to 7631
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                7088 non-null   int64  
 1   LocID                     7088 non-null   int64  
 2   Location                  7088 non-null   object 
 3   Time Period               7088 non-null   object 
 4   MidPeriod                 7088 non-null   int64  
 5   Total Fertility           7088 non-null   float64
 6   NRR(surviving daughters)  7088 non-null   float64
 7   Crude Birth Rate          7088 non-null   float64
 8   Births                    7088 non-null   float64
 9   Life Expectancy(birth)    7088 non-null   float64
 10  LExMale                   7088 non-null   float64
 11  LExFemale                 7088 non-null   float64
 12  Infant Mortality Rate     7088 non-null   float64
 13  Under-five Mortality      7088 non-null   float64
 14  Crude De

In [6]:
df.corr()

,Unnamed: 0,LocID,MidPeriod,Total Fertility,NRR(surviving daughters),Crude Birth Rate,Births,Life Expectancy(birth),LExMale,LExFemale,Infant Mortality Rate,Under-five Mortality,Crude Death Rate,Deaths,DeathsMale,DeathsFemale,Net Migration Rate(pK),Net Migrants(K),GrowthRate,Natural Increase Rate,Sex ratio(m per f births,Mean Age Childbearing(f),PopFemale,PopMale,PopTotal,PopDensity
Unnamed: 0,1.000000,1.150010e-01,1.882997e-03,0.035619,0.049049,0.027552,0.083345,-0.007368,-0.008240,-0.005441,0.001786,0.002113,-0.004780,0.079705,0.079601,0.079773,0.009614,0.000467,0.036884,0.036352,-0.040937,0.030808,0.078898,0.076674,0.077779,-0.053888
LocID,0.115001,1.000000e+00,6.678162e-17,-0.018191,-0.042183,-0.006990,0.349902,-0.050572,-0.048896,-0.051980,0.074841,0.061842,0.022538,0.339482,0.339806,0.338917,0.012125,-0.075898,-0.010909,-0.020365,0.171330,-0.066170,0.326534,0.324994,0.325780,-0.087305
MidPeriod,0.001883,6.678162e-17,1.000000e+00,-0.536580,-0.488129,-0.577177,0.024017,0.657374,0.676729,0.636959,-0.592875,-0.558928,-0.350596,0.124363,0.123335,0.125432,0.010903,-0.037130,-0.438558,-0.526010,0.060118,0.239992,0.171441,0.169626,0.170536,0.066422
Total Fertility,0.035619,-1.819134e-02,-5.365795e-01,1.000000,0.947370,0.983822,-0.026479,-0.878907,-0.862902,-0.887643,0.858983,0.858352,0.632857,-0.078312,-0.080097,-0.076276,-0.050887,-0.095700,0.713911,0.878185,-0.366239,0.336256,-0.161623,-0.158142,-0.159875,-0.112642
NRR(surviving daughters),0.049049,-4.218319e-02,-4.881295e-01,0.947370,1.000000,0.926462,-0.046548,-0.729365,-0.714381,-0.737235,0.681679,0.663777,0.379474,-0.111123,-0.112176,-0.109884,-0.049454,-0.103613,0.767387,0.940102,-0.356688,0.305017,-0.174548,-0.171272,-0.172905,-0.114052
Crude Birth Rate,0.027552,-6.989685e-03,-5.771770e-01,0.983822,0.926462,1.000000,-0.005320,-0.910672,-0.897149,-0.917208,0.874550,0.871301,0.609647,-0.075568,-0.076566,-0.074409,-0.073771,-0.127895,0.727406,0.910191,-0.358069,0.255076,-0.150655,-0.146758,-0.148696,-0.112709
Births,0.083345,3.499024e-01,2.401737e-02,-0.026479,-0.046548,-0.005320,1.000000,-0.067764,-0.054524,-0.078954,0.085547,0.070177,-0.019299,0.922801,0.927863,0.916606,-0.003621,-0.377816,0.000301,0.003547,0.331068,-0.124402,0.922531,0.924822,0.923757,-0.038314
Life Expectancy(birth),-0.007368,-5.057234e-02,6.573743e-01,-0.878907,-0.729365,-0.910672,-0.067764,1.000000,0.996916,0.997463,-0.949806,-0.950813,-0.773516,-0.002939,-0.001851,-0.004149,0.081298,0.125577,-0.557435,-0.714805,0.284401,-0.122073,0.085866,0.082442,0.084141,0.126880
LExMale,-0.008240,-4.889595e-02,6.767288e-01,-0.862902,-0.714381,-0.897149,-0.054524,0.996916,1.000000,0.988903,-0.939478,-0.939908,-0.769638,0.010209,0.011214,0.009082,0.085915,0.112837,-0.542380,-0.700215,0.289666,-0.090666,0.096811,0.094060,0.095427,0.129967
LExFemale,-0.005441,-5.197960e-02,6.369594e-01,-0.887643,-0.737235,-0.917208,-0.078954,0.997463,0.988903,1.000000,-0.954617,-0.956178,-0.774250,-0.014487,-0.013277,-0.015825,0.078383,0.135599,-0.565610,-0.722453,0.279231,-0.149532,0.076107,0.072141,0.074107,0.123035


In [7]:
# baseline regression model (from previous assignment)
baseline = round(df.Births.mean())
baselist = [baseline] * len(df.Births)
errs = baseline - df['Births']
mae = errs.abs().mean()
print(f'The baseline is {baseline:,.0f} births,')
print(f'with a mean absolute error of {mae:,.0f}.')

The baseline is 57,279 births,
with a mean absolute error of 72,921.


In [8]:
# Train/Val/Test split
train,Test = train_test_split(df, train_size=0.80, test_size=0.20)
Train, Val = train_test_split(train,train_size=0.80, test_size=0.20)
Train.shape,Val.shape,Test.shape

((4536, 28), (1134, 28), (1418, 28))

## **Intro:**

In [9]:
# In the first assignment I chose 'Births as my target
target = 'Births'
# The features chosen 
features = ['Total Fertility','GrowthRate','Crude Birth Rate','Natural Increase Rate','PopFemale','PopTotal']

In [10]:
train.columns
drop = ['Unnamed: 0','Location','Time Period', target]
X_train = train.drop(columns=drop)
X_test = Test.drop(columns=drop)
Y_train = train[target]

In [12]:
# I became curious about which features SelectKbest would choose
select = SelectKBest(k=10)


x_trainS = select.fit_transform(X_train, Y_train)
x_testS = select.transform(X_test)
selected_mask = select.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

Features selected:
MidPeriod
Deaths
DeathsMale
DeathsFemale
Net Migrants(K)
Mean Age Childbearing(f)
PopFemale
PopMale
PopTotal
PopDensity


In [13]:
# X and Y
x_train = Train[features]
x_val   = Val[features]
x_test  = Test[features]

y_train = Train[target]
y_val   = Val[target]
y_test  = Test[target]